## **Identificação de documentos similares/relevantes para cancro do estômago**

 **Projeto C.3 - Aprendizagem Automática 2**

**Orientadores : Rúben Rodrigues e Nuno Alves**

 ## **Contextualização**

Repositórios como o PUBMED contêm mais de 30 milhões de documentos que estão indexados a termos chave.
Estes repositórios possibilitam a identificação de documentos relevantes sobre um determinado termo.

 **Problema**

Existem documentos que são relevantes a um determinado tópico, mas que não estão devidamente identificados.

 **Solução**

Existem abordagens baseadas em aprendizagem máquina não supervisionadas como a Latent Dirichlet Allocation (LDA) em que é possível aplicar o cálculo da distância Jensen-Shannon para identificar documentos similares a um determinado tópico. Exemplos de pesquisas que podem beneficiar de um sistema orientado a similaridade de documentos incluem a pesquisa sobre uma determinada doença como o cancro do estômago e um tratamento que não foi indexado pelo repositório.

 **Objetivo**

Implementar uma pipeline em python que calcule a similaridade de documentos recorrendo ao LDA e à fórmula de Jensen-Shannon e integrem a pipeline desenvolvida na framework do grupo BioSystems (BioTMpy).

 ## **Criação do dataset**

Os dados fornecidos para este trabalho estarão no formato JSON e serão disponibilizados a partir da plataforma de conhecimento sobre cancro gástrico desenvolvido no grupo BioSystems.

**Aceder à API e analisar a estrutura dos dados**

Iniciamos o projeto por aceder á API (https://gcplatform.bio.di.uminho.pt/api/pub/swagger-ui/index.html?configUrl=/api/pub/v3/api-docs/swagger-config#/pageable-publications-controller/findPubmedPubsByQuery) e analisar os dados relativos ao cancro do estômago.

Após uma análise verificamos que a estrutura dos objetos era a seguinte:

{
    id (string)
    idGenerator (string)
    source (string)
    createdDate (string date-time)
    lastModifiedDate (string date-time)
    title (string)
    summary (string)
    content (string)
    xdbRefs (uniqueItems: true string)
    meshTerms (uniqueItems: true string)
    keywords (uniqueItems: true string)
    
}

Também verificamos que da estrutura das páginas JSON da API apenas nos interessava o array "content", que continha os objetos com a estrutura em cima representada e com a informação que necessitavamos.





Atenção, a execução deste notebook não é rápida por isso caso pretendam aceder aos ficheiros resultantes da mesma podem encontrá-los nos respetivos links.

Todos os documentos em baixo descritos : https://we.tl/t-LV67ORGzZm 

Pasta pages com os ficheiros json de cada página : https://we.tl/t-wYMoIfvL3p

Ficheiro csv com o dataset : https://we.tl/t-vR3WrUjWnI

Ficheiro json com todos os objetos da API : https://we.tl/t-Av1CzQBMQh



**Recolher informação da API para um ficheiro json**

De seguida, foi necessário recolher a informação importante da API para um ficheiro JSON.
Para tal, utilizamos três bibliotecas. A biblioteca **urllib.parse**, para manipularmos o url, a biblioteca **requests**, que é uma biblioteca HTTP necessária para recebermos a informação da API, e, a biblioteca **json**, para manipular os ficheiros json.

In [ ]:
import urllib.parse, requests, json

Posto isto, criamos um loop que efetuas pedidos sobre as 145 páginas existentes na API sobre o cancro do estômago e que coloca, a informação do content de cada página, numa pasta.

In [ ]:
#Rescolher a informação da API pub
for n_page in range(146):
    main_api = 'https://gcplatform.bio.di.uminho.pt/api/pub/page/find/pubmedindexed/gastric%20cancer?page={}&size=1000&sort=string'.format(n_page)
    address = 'lhr'
    url = main_api + urllib.parse.urlencode({'address': address})
    json_data = requests.get(url).json()
    print(json_data["content"])

    #Mudar o nome do ficheiro
    filename = "C:\\Users\\Simon\\OneDrive\\Documentos\\GitHub\\projeto_aa2\\pages\\page_{}_gastric_cancer.json".format(n_page)
  

    #Escrever a informação num ficheiro json
    with open(filename, 'a') as json_file:
        json.dump(json_data["content"], json_file, indent=4, sort_keys=True) #Formata o json file, o indent e sort_keys
        

Por fim, foi necessário juntar todas as páginas numa única lista de objetos, ou seja, num ficheiro json com todos os objetos.

In [ ]:
import json
from os import listdir
from os.path import isfile, join


lista=[]  


files = [f for f in listdir("pages") if isfile(join("pages", f))]

for file in files:
    f = open('pages/'+file)
    data = json.load(f) 
    for i in data: lista.append(i)
    f.close()

print(lista)

with open('info_gastric_cancer.json', 'w') as json_file:
    json.dump(lista, json_file,indent=1)

**Converter o ficheiro json em objetos python**

Inicialmente, criamos uma classe Info com quatro métodos, sendo dois os mais importantes. O primeiro método, o init, é executado quando criamos uma instância de um objeto. O segundo método, o to dict, é utilizado para converter os objetos que guardamos na lista num dicionário.

In [ ]:
import json
import pandas as pd


class Info:
    def __init__(self, id=None, idGenerator=None, source=None, createdDate=None, lastModifiedDate=None, title=None, summary=None, content=None, xdbRefs=None, meshTerms=None, keywords=None):
        self.id = id
        self.idGenerator = idGenerator
        self.source = source
        self.createdDate = createdDate
        self.lastModifiedDate = lastModifiedDate
        self.title = title
        self.summary = summary
        self.content = content
        self.xdbRefs = xdbRefs
        self.meshTerms = meshTerms
        self.keywords = keywords


    @classmethod
    def from_json(cls, json_string):
        json_dict = json.loads(json_string)
        return cls(**json_dict)


    def __repr__(self):
        #return str(self)
        return f'{self.id}'
        
    def to_dict(self):
        return {
            'id': self.id,
            'title': self.title,
            'summary':self.summary,
        }

Posto isto, foi necessário ler o ficheiro json com todos os objetos e colocar esses objetos numa lista.

In [ ]:
info_list = []
with open('info_gastric_cancer.json', 'r') as json_file:
    info_data = json.loads(json_file.read())
    for i in info_data: 
        info_list.append(Info(**i))

**Converter os objetos python para pandas**

Tendo já uma lista com os objetos é necessário colocar estes objetos num dataframe, recorrendo à biblioteca pandas.

In [ ]:
df = pd.DataFrame.from_records([a.to_dict() for a in info_list])
df.index = df["id"]
del df["id"]

**Exportar para csv**

Por fim, foi exportado o dataframe para um ficheiro csv.

In [ ]:
df.to_csv('dataset_gastric_cancer.csv', sep=",")